# Span Detection UA only implementation

1. Import dependencies

In [ ]:
import random
import time

import numpy as np
import torch
import os

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

from src.util.torch_device import resolve_torch_device
from src.data.span_detection_ds import ManipulationDetectionDataset
from src.visualization.plot import plot_loss, plot_model_progress, plot_eval_loss
from src.definitions import (
    MODELS_FOLDER,
    RAW_DATA_FOLDER,
    REPORTS_FOLDER,
    PROCESSED_DATA_FOLDER,
)
from src.visualization.ner import MarkdownVisualizer, VisualizationMode
from src.visualization.reporting import EvaluatingReport
from src.model.span_detection_metrics import compute_metrics

2. Prepare Env

In [ ]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

model_checkpoint = MODELS_FOLDER / "ua-fine-tuned-xlm-roberta-large"

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

3. Load dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset_blueprint = ManipulationDetectionDataset(
    tokenizer=tokenizer,
    raw_path=RAW_DATA_FOLDER / "span-detection.parquet",
    processed_path=PROCESSED_DATA_FOLDER / "span-detection-ua" / model_checkpoint,
    seed=random_seed,
    lang="uk",
)

dataset = dataset_blueprint.read()

3. Prepare model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(dataset_blueprint.label2id),
    id2label=dataset_blueprint.id2label,
    label2id=dataset_blueprint.label2id,
    classifier_dropout=0.1,
    ignore_mismatched_sizes=True
).to(device)

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=MODELS_FOLDER / "manipulation-span-detection-ua-checkpoint",
    learning_rate=3e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=random_seed,
    logging_steps=100,
    auto_find_batch_size=True,
    torch_empty_cache_steps=1000,
    metric_for_best_model="token_f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics(dataset_blueprint),
)

4. Train model

In [ ]:
trainer.train()

In [ ]:
evaluation_feedback = trainer.evaluate()

In [ ]:
report_file = REPORTS_FOLDER / "span-detection-ua" / "train-report.csv"

report = EvaluatingReport(report_file)

report.write_to_report(evaluation_feedback, epoch_time)

full_report = report.read_report()

full_report.tail(1)

In [ ]:
plot_loss(trainer)

In [ ]:
plot_eval_loss(trainer)

In [ ]:
plot_model_progress(full_report)

5. Save weights

In [ ]:
trainer.save_model(MODELS_FOLDER / "manipulation-span-detection-ua-model")
tokenizer.save_pretrained(MODELS_FOLDER / "manipulation-span-detection-ua-model")

6. Test

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    MODELS_FOLDER / "manipulation-span-detection-ua-model"
)
tokenizer = AutoTokenizer.from_pretrained(MODELS_FOLDER / "manipulation-span-detection-ua-model")

In [ ]:
test_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

test_result = test_pipeline(dataset["test"]["content"])

In [ ]:
visualizer = MarkdownVisualizer(
    tokenizer=tokenizer,
    path=REPORTS_FOLDER
    / "span-detection-ua"
    / "test-visualization"
    / f"test-{epoch_time}.md",
    visualization_mode=VisualizationMode.ROBERTA,
)

visualizer.visualize_as_markdown_and_save(dataset["test"], test_result)